In [3]:
import ee
#ee.Authenticate()

ee.Initialize(project="myproject-480016")

In [4]:
dem = ee.Image("USGS/3DEP/10m")

geom = ee.Geometry.Point([-91.0989573,30.3529013])

geom_col = ee.FeatureCollection([geom])
elev = dem.sampleRegions(geom_col).getInfo()
elev["features"][0]["properties"]["elevation"]

c:\Users\jemad\path_name\arcpy_clone2\Lib\site-packages\ee\deprecation.py:215: DeprecationWarning: 

Attention required for USGS/3DEP/10m! You are using a deprecated asset.
To make sure your code keeps working, please update it.
This dataset has been superseded by USGS/3DEP/10m_collection

Learn more: https://developers.google.com/earth-engine/datasets/catalog/USGS_3DEP_10m

  warnings.warn(warning, category=DeprecationWarning)


3.220296621322632

In [5]:
import pandas as pd
table = pd.read_csv(r"C:\Users\jemad\Documents\Documents\LSU\4th Semester_Fall 2025\GEOG 4057_GIS Programming\geog4057-main\project\data\boundary.csv")

In [6]:
import arcpy
ra1 = arcpy.Raster(r"C:\Users\jemad\Documents\Documents\LSU\4th Semester_Fall 2025\GEOG 4057_GIS Programming\geog4057-main\project\data\flood_2class.tif") 
print(ra1.spatialReference.factoryCode)

32119


In [7]:
import geopandas
gdf = geopandas.GeoDataFrame(table)

gdf.set_geometry(geopandas.points_from_xy(gdf['X'], gdf['Y']),
                 inplace=True, crs=f'EPSG:{ra1.spatialReference.factoryCode}')


In [8]:
from osgeo import gdal
print(gdal.__version__)

3.10.2e


In [9]:
gdf.to_file(r"C:\Users\jemad\Documents\Documents\LSU\4th Semester_Fall 2025\GEOG 4057_GIS Programming\geog4057-main\project\data\boundary.shp") 

In [10]:
shapefile = r"C:\Users\jemad\Documents\Documents\LSU\4th Semester_Fall 2025\GEOG 4057_GIS Programming\geog4057-main\project\data\boundary.shp"
arcpy.management.AddField(shapefile, "elevation", field_type='FLOAT')

<Result 'C:\\Users\\jemad\\Documents\\Documents\\LSU\\4th Semester_Fall 2025\\GEOG 4057_GIS Programming\\geog4057-main\\project\\data\\boundary.shp'>

In [11]:
geom_list = []
with arcpy.da.SearchCursor(shapefile,['SHAPE@XY'],spatial_reference=4326) as cursor:
    for row in cursor:
        X,Y = row[0]
        geom = ee.Geometry.Point([X,Y])
        geom_list.append(geom)
geom_col = ee.FeatureCollection(geom_list)
elev = dem.sampleRegions(geom_col).getInfo().get('features')


In [12]:
i = 0
with arcpy.da.UpdateCursor(shapefile,['elevation']) as cursor:
    for row in cursor:
        elevation = elev[i]['properties']['elevation']
        row[0] = elevation
        cursor.updateRow(row)
        i += 1